In [1]:
import os
os.environ["TRANSFORMERS_NO_TF_IMPORT"] = "1"
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling


c:\Users\hp\anaconda3\envs\crypto_nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cuda pruefen
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


Model

In [3]:
# 📌 Pfade zu den CSV-Dateien
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts_rohdaten.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments_rohdaten.csv")

# 🔹 **Funktion zum Laden der CSV-Dateien**
def load_csv(filepath):
    """Lädt eine CSV-Datei mit `|` als Trennzeichen und Debugging-Infos"""
    if not os.path.exists(filepath):
        print(f"❌ Datei nicht gefunden: {filepath}")
        return pd.DataFrame()

    df = pd.read_csv(filepath, sep="|", encoding="utf-8-sig", on_bad_lines="skip")

    print(f"\n📌 Datei geladen: {filepath}")
    print(f"🔹 Spalten: {df.columns.tolist()}")
    print(df.dtypes)
    print(df.head())

    return df

# 📌 **Daten laden**
df_posts = load_csv(POSTS_CSV)
df_comments = load_csv(COMMENTS_CSV)


📌 Datei geladen: G:/Meine Ablage/reddit/reddit_posts_rohdaten.csv
🔹 Spalten: ['post_id', 'crypto', 'search_term', 'subreddit', 'title', 'author', 'date', 'time', 'score', 'num_comments', 'selftext']
post_id         object
crypto          object
search_term     object
subreddit       object
title           object
author          object
date            object
time            object
score            int64
num_comments     int64
selftext        object
dtype: object
   post_id   crypto search_term       subreddit  \
0  1j0dz4r  Bitcoin     bitcoin  CryptoCurrency   
1  1j0dghm  Bitcoin     bitcoin  CryptoCurrency   
2  1j0cx0g  Bitcoin     bitcoin  CryptoCurrency   
3  1j0cnlh  Bitcoin     bitcoin  CryptoCurrency   
4  1j08i0y  Bitcoin     bitcoin  CryptoCurrency   

                                               title               author  \
0  BlackRock Adds Its Bitcoin ETF to Model Portfo...             diwalost   
1  x-post: As the free-float of coins is low, is ...        3fkgf9fmd980

In [4]:
df_posts.date.max(), df_posts.date.min()

('2025-02-28', '2024-11-01')

In [10]:
# 🔄 2️⃣ Relevante Spalten extrahieren
df_posts["full_text"] = df_posts["title"].fillna("") + " " + df_posts["selftext"].fillna("")
df_comments["full_text"] = df_comments["selftext"].fillna("")

# ❌ Entferne leere Zeilen
df_posts = df_posts[df_posts["full_text"].str.strip() != ""]
df_comments = df_comments[df_comments["full_text"].str.strip() != ""]

# Entferne sehr kurze Texte (< 10 Zeichen) und doppelte Einträge
df_posts = df_posts[df_posts["full_text"].str.len() > 10].drop_duplicates(subset=["full_text"])
df_comments = df_comments[df_comments["full_text"].str.len() > 10].drop_duplicates(subset=["full_text"])

# 🔄 3️⃣ Posts & Kommentare kombinieren
df = pd.concat([df_posts[["full_text"]], df_comments[["full_text"]]], ignore_index=True)

print(f"✅ Bereinigung abgeschlossen. {len(df)} Einträge gespeichert in reddit_cleaned.csv")


✅ Bereinigung abgeschlossen. 334346 Einträge gespeichert in reddit_cleaned.csv


In [11]:
# 🔹 Stichprobe ziehen (z. B. 20.000 Einträge)
df_sample = df.sample(n=20000, random_state=42)

# 🔹 In Hugging Face Dataset umwandeln
dataset = Dataset.from_pandas(df_sample)

In [12]:
# 🔠 2️⃣ Tokenizer laden (BERT als Basis für CryptoBERT)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenisierung mit Padding & Truncation
def tokenize_function(examples):
    return tokenizer(examples["full_text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["full_text"])

# 🔹 3️⃣ Masked Language Modeling (MLM) vorbereiten
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 🔹 4️⃣ CryptoBERT-Modell laden (auf BERT-Basis)
model = BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)

# 🔹 5️⃣ Trainingseinstellungen
training_args = TrainingArguments(
    output_dir="./cryptoBERT-posttrained",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none"
)

# 🔹 6️⃣ Trainer definieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 🔥 7️⃣ Training starten
trainer.train()

# 📌 8️⃣ Modell speichern (nach jeder Epoche wird es automatisch gespeichert)
trainer.save_model("./cryptoBERT-posttrained-final")
print("✅ Training abgeschlossen. Modell gespeichert!")

Map: 100%|██████████| 20000/20000 [00:10<00:00, 1847.91 examples/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\hp\anaconda3\envs\crypto_nlp\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:

Epoch,Training Loss,Validation Loss
1,2.653000,2.427955
2,2.392600,2.259322


✅ Training abgeschlossen. Modell gespeichert!


In [15]:
import torch
import math
from transformers import BertTokenizer, BertForMaskedLM

# 📌 Lade das trainierte CryptoBERT-Modell
crypto_model_path = "cryptoBERT-posttrained-final"
crypto_model = BertForMaskedLM.from_pretrained(crypto_model_path).eval()

# 📌 Lade das ursprüngliche BERT-Modell zum Vergleich
base_model = BertForMaskedLM.from_pretrained("bert-base-uncased").eval()

# 📌 Tokenizer laden
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 📌 Beispieltext aus den Reddit-Daten
text = "Bitcoin is pumping 🚀. The market is looking bullish today."

# 🔠 Tokenisierung
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# 🔥 Perplexity-Funktion
def calculate_perplexity(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        ppl = math.exp(loss.item())
        return ppl

# 📊 Berechnung des PPL-Scores für beide Modelle
crypto_ppl = calculate_perplexity(crypto_model, input_ids)
base_ppl = calculate_perplexity(base_model, input_ids)

print(f"📊 Perplexity Score (Original BERT): {base_ppl:.2f}")
print(f"🚀 Perplexity Score (CryptoBERT trainiert): {crypto_ppl:.2f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


📊 Perplexity Score (Original BERT): 8.73
🚀 Perplexity Score (CryptoBERT trainiert): 6.86


In [20]:
import torch
import math
from transformers import AutoTokenizer, AutoModelForMaskedLM

# 📌 Modelle direkt von Hugging Face laden
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased").eval()
crypto_bert_base = AutoModelForMaskedLM.from_pretrained("ElKulako/cryptobert").eval()
crypto_trained_model = AutoModelForMaskedLM.from_pretrained("cryptoBERT-posttrained-final").eval()

# 📌 Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 📌 Beispieltext aus Reddit (Krypto-Sprache)
text = "Bitcoin is pumping 🚀. The market is looking bullish today."

# 🔠 Tokenisierung
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# 🔥 Perplexity-Funktion
def calculate_perplexity(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        ppl = math.exp(loss.item())
        return ppl

# 📊 Berechnung des PPL-Scores für alle Modelle
ppl_base = calculate_perplexity(base_model, input_ids)
ppl_crypto_bert = calculate_perplexity(crypto_bert_base, input_ids)
ppl_trained = calculate_perplexity(crypto_trained_model, input_ids)

print(f"📊 Perplexity Score (Original BERT): {ppl_base:.2f}")
print(f"📊 Perplexity Score (CryptoBERT von Hugging Face - ElKulako): {ppl_crypto_bert:.2f}")
print(f"🚀 Perplexity Score (Unser trainiertes CryptoBERT): {ppl_trained:.2f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ElKulako/cryptobert and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model o

📊 Perplexity Score (Original BERT): 8.73
📊 Perplexity Score (CryptoBERT von Hugging Face - ElKulako): 2166877.11
🚀 Perplexity Score (Unser trainiertes CryptoBERT): 6.86


In [21]:
from transformers import AutoModel, AutoTokenizer

# 📌 Modell & Tokenizer laden
MODEL_NAME = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# 📌 Modellarchitektur überprüfen
print(f"✅ Modell erfolgreich geladen: {MODEL_NAME}")
print(f"🔍 Modellarchitektur: {model.config.architectures}")
print(f"🔢 Anzahl der Modellparameter: {model.num_parameters()}")


Some weights of RobertaModel were not initialized from the model checkpoint at ElKulako/cryptobert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modell erfolgreich geladen: ElKulako/cryptobert
🔍 Modellarchitektur: ['RobertaForSequenceClassification']
🔢 Anzahl der Modellparameter: 124645632
